# Neurosymbolic Software Tutorial - NEAR Bouncing Ball

<a target="_blank" href="https://colab.research.google.com/github/kavigupta/neurosym-lib/blob/main/tutorial/bouncing_ball_exercise_skeleton.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Instruction
- Navigating this notebook on Google Colab: There will be text blocks and code blocks throughout the notebook. The text blocks, such as this one, will contain instructions and questions for you to consider. The code blocks, such as the one below, will contain executible code. Sometimes you will have to modify the code blocks following the instructions in the text blocks. You can run the code block by either pressing control/cmd + enter or by clicking the arrow on left-hand side.
- Saving Work: If you wish to save your work in this .ipynb, we recommend downloading the compressed repository from GitHub, unzipping it, uploading it to Google Drive, and opening this notebook from within Google Drive.

## Notebook

In this notebook, you will construct a DSL to simulate a bouncing ball

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import neurosym as ns
from neurosym.examples import near

pl = ns.import_pytorch_lightning()


In [2]:
torch.manual_seed(0)
np.random.seed(0)

## Data

We then load and plot some bouncing ball trajectories. Note that these trajectories are represented as a list `[x, y, vx, vy]`

In [3]:
dataset_factory = lambda train_seed: ns.DatasetWrapper(
    ns.DatasetFromNpy(
        f"bouncing_ball_exercise/data/bounce_example/train_ex_data.npy",
        f"bouncing_ball_exercise/data/bounce_example/train_ex_labels.npy",
        train_seed,
    ),
    ns.DatasetFromNpy(
        f"bouncing_ball_exercise/data/bounce_example/test_ex_data.npy",
        f"bouncing_ball_exercise/data/bounce_example/test_ex_labels.npy",
        None,
    ),
    batch_size=200,
)
datamodule = dataset_factory(42)

In [4]:
def plot_trajectory(trajectory, color):
    plt.scatter(trajectory[:, 0], trajectory[:, 1], marker="o", color=color)
    plt.plot(trajectory[:, 0], trajectory[:, 1], alpha=0.2, color=color)
    plt.xlim(-5, 10)
    plt.ylim(-5, 7)
    plt.grid(True)

In [5]:
for i in range(3):
    plot_trajectory(datamodule.train.inputs[i], f"C{i}")

In [6]:
print("input[0] :", datamodule.train.inputs[i, 0])
print("output[0]:", datamodule.train.outputs[i, 0])

## Exercise: DSL

Fill in the `bounce_dsl` to parameterize the space of functions that could represent the trajectories of bouncing balls.

In [7]:
def bounce_dsl():
    L = 4
    O = 4
    dslf = ns.DSLFactory(L=L, O=O, max_overall_depth=5)
    "YOUR CODE HERE"
    dslf.prune_to("[$fL] -> [$fL]")
    return dslf.finalize()

In [8]:
dsl = bounce_dsl()

### DSL Printout

See your DSL printed below, and ensure it is what you would expect

In [9]:
print(dsl.render())

### Setting up Neural DSL

In [10]:
input_dim, output_dim = 4, 4
max_depth = 16

In [11]:
t = ns.TypeDefiner(L=input_dim, O=output_dim)
t.typedef("fL", "{f, $L}")
neural_modules = {
        **near.create_modules(
            "mlp",
            [t("($fL) -> $fL")],
            near.mlp_factory(hidden_size=10),
        ),
        **near.create_modules(
            "rnn_seq2seq",
            [t("([$fL]) -> [$fL]")],
            near.rnn_factory_seq2seq(hidden_size=10),
        ),
    }

In [12]:
interface = near.NEAR(
    input_dim=input_dim,
    output_dim=output_dim,
    max_depth=max_depth,
    lr=5e-3,
    max_seq_len=300,
    n_epochs=100,
    accelerator="cpu",
)
interface.register_search_params(
    dsl=dsl,
    type_env=t,
    neural_modules=neural_modules,
    search_strategy=ns.search.bounded_astar,
    loss_callback=torch.nn.functional.mse_loss,
    validation_params=dict(
        enable_progress_bar=False,
        enable_model_summary=False,
        progress_by_epoch=True,
    )
)

### Run NEAR

In [13]:
best_programs = interface.fit(
    datamodule=datamodule,
    program_signature="([{f, $L}]) -> [{f, $O}]",
    n_programs=4
)

### Top 3 Programs

The code below assumes you found some top 3 programs and stored them in the best_program_nodes variable.

The function below is set up to further fine tune the program, test it, and return a set of values produced by it.

In [14]:
best_program_nodes = [(p, None) for p in best_programs]

In [15]:
def testProgram(best_program_node):
    module = interface.train_program(
        best_program_node[0].program,
        datamodule,
        max_epochs=4000,
        callbacks=[
            pl.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5)
        ],
        enable_progress_bar=False,
    )
    T = 100
    path = np.zeros((T, 4))
    X = torch.tensor(
        np.array([0.21413583, 4.4062634, 3.4344807, 0.12440437]), dtype=torch.float32
    )
    for t in range(T):
        path[t, :] = X.detach().numpy()
        Y = best_program(X.unsqueeze(0)).squeeze(0)
        X = Y
    return path

In [16]:
trajectories = [program_to_trajectory(prog) for prog in best_programs]

## Plotting Trajectories

In [17]:
plt.figure(figsize=(8, 8))

for i, trajectory in enumerate(trajectories):
    plot_trajectory(trajectory, f"C{i}")
plot_trajectory(datamodule.train.inputs[0], "black")

plt.title("Bouncing ball (ground truth in black)")
plt.show()
